<a href="https://colab.research.google.com/github/andrawaag/BioHackathon/blob/master/notebooks/Wikibase_properties.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install libraries not yet install in colab

In [0]:
!pip install wikidataintegrator

     |████████████████████████████████| 71kB 4.6MB/s 
     |████████████████████████████████| 92kB 7.6MB/s 
     |████████████████████████████████| 51kB 5.8MB/s 
     |████████████████████████████████| 235kB 14.5MB/s 
     |████████████████████████████████| 112kB 20.1MB/s 
     |████████████████████████████████| 71kB 8.2MB/s 
     |████████████████████████████████| 61kB 7.8MB/s 
     |████████████████████████████████| 61kB 8.3MB/s 
     |████████████████████████████████| 51kB 8.1MB/s 
     |████████████████████████████████| 112kB 17.2MB/s 
  Created wheel for wikidataintegrator: filename=wikidataintegrator-0.6.0-cp36-none-any.whl size=73229 sha256=a7306bca36169e87c6f92d45266255f3869357bc3cb1150cadb1396f6d783b27
  Stored in directory: /root/.cache/pip/wheels/40/94/f8/8815e5f953d2b19db82c278303fc8cf7395116b5cb5cc30872
  Created wheel for simplejson: filename=simplejson-3.17.0-cp36-cp36m-linux_x86_64.whl size=114207 sha256=40134bde3f43922f2550f83fa7428b6d0df605eeca65934d7ed5c7a503edaf3c
 

# Configure this notebook

In [0]:
from getpass import getpass
from wikidataintegrator import wdi_core, wdi_login
import pprint

wikibase = "https://sandbox.wiki.opencura.com/" ## <-- Add wikibase main host name
api = wikibase+"w/api.php"
sparql = wikibase+"query/sparql"
entityUri = wikibase.replace("https:", "http:")+"entity/"

WBUSER = getpass(prompt="username:")  
WBPASS = getpass(prompt='Enter your password: ')  
login = wdi_login.WDLogin(WBUSER, WBPASS, mediawiki_api_url=api)



username:··········
Enter your password: ··········
https://jacana-jacana.semscape.org/w/api.php
Successfully logged in as Andrawaag


In [0]:
query = "PREFIX wd: <"+entityUri+"> "
query += """
SELECT DISTINCT ?property ?propertyLabel ?propertyType WHERE {
  ?property rdf:type wikibase:Property ;
            wikibase:propertyType ?propertyType ;
            rdfs:label ?propertyLabel .
}
"""

results = wdi_core.WDItemEngine.execute_sparql_query(query, endpoint=sparql)
properties = dict()
for property in results["results"]["bindings"]:
  properties[property["propertyLabel"]["value"].lower()] = property["property"]["value"]

def createProperty(login, label, description, property_datatype):
  s = []
  localEntityEngine = wdi_core.WDItemEngine.wikibase_item_engine_factory(mediawiki_api_url=api, sparql_endpoint_url=sparql)
  item = localEntityEngine(data=s)
  item.set_label(label)
  item.set_description(description)
  return item.write(login, entity_type="property", property_datatype=property_datatype)

  properties

# Select CSV file with header
Here we upload a csv file that contains a header with the field names. These field names will be later compared with the labels that are available in the wikibase under scrutiny. The CSV file be uploaded and stored as a pandas dataframe. 

In [0]:
from google.colab import files
import pandas as pd
results = files.upload()

print(results)
for key in results.keys():
  csv  = pd.read_csv(key, sep='\t',chunksize=100) 
 
csv

Saving rights.txt to rights.txt
{'rights.txt': b'\nDataset: Provincial Museum of Alberta, Edmonton, AB, Canada. Birds (Aves)\nRights as supplied: http://creativecommons.org/licenses/by/4.0/legalcode\nDataset: Aves MZS\nRights as supplied: http://creativecommons.org/licenses/by/4.0/legalcode\nDataset: Collections and observation data National Museum of Natural History Luxembourg\nRights as supplied: http://creativecommons.org/publicdomain/zero/1.0/legalcode\nDataset: Collection Aves (bird skins) SMF\nRights as supplied: http://creativecommons.org/licenses/by/4.0/legalcode\nDataset: Aves de la expedici\xc3\xb3n en lago de Tota, Boyac\xc3\xa1 - Proyecto Boyac\xc3\xa1 BIO\nRights as supplied: http://creativecommons.org/licenses/by-nc/4.0/legalcode\nDataset: MSB Bird Collection (Arctos)\nRights as supplied: http://creativecommons.org/publicdomain/zero/1.0/legalcode\nDataset: Biodiversidad de un bosque seco tropical del Valle Medio del r\xc3\xado Magdalena: evaluaci\xc3\xb3n integrada a trav

# Dashboard to create properties
Here a Dasboard is created from where the properties will be created in the Wikibase under scrutiny. The dashboard will have 4 columns. The first column will have buttons. If the button is blue, a property with the same label already exists. If it is orange, the property is created during the same session. If the button is green, there is not yet a property on the Wikibase under scrutiny with that labels.

## Caveats
1. **Expects \t as delimiter**. This scripts assumes the uploaded csv file uses the tab (\t) as the delimiter. Future adaptations of this script will include the choice to specify the delimiter
1. **Only label matching.**
This code does only check for existing properties with the exact same label. Any slide difference in spelling will result in this script creating a new property, while a similar property exists. I have created this script to create properties in an empty wikibase. Future adaptations might have more rules in place to identify existing properties with spelling differences
2. **Datatypes are ignored.** This script ignores datatypes both in the source (i.e. uploaded csv file) as in the wikibase under scrutiny



In [0]:

from ipywidgets import interact, interactive, fixed, interact_manual, GridspecLayout

import ipywidgets as widgets

grid = GridspecLayout(len(list(csv))+1, 4)

print(len(list(csv)))

grid[0,0]= widgets.Label("Add to Wikibase")
grid[0,1]= widgets.Label("Property name")
grid[0,2]= widgets.Label("Example value")
grid[0,3]= widgets.Label("Property value type")

display(grid)
property = dict()
i = 1

class PropertyRow: 
   def __init__(self, field=''): 
     self.field = field    
     if self.field.lower() not in properties.keys():
        self.PropertyButton =  widgets.Button(
          description='Create',
          disabled=False,
          button_style='success', # 'success', 'info', 'warning', 'danger' or ''
          tooltip=self.field,
          icon='check', # (FontAwesome names without the `fa-` prefix)
          field=self.field
          )
        self.PropertyButton.on_click(self.on_button_clicked) 
     else:
        self.PropertyButton =  widgets.Button(
          description= properties[field.lower()].replace(wikibase.replace("https", "http")+"entity/","").replace("https", "http"),
          disabled=False,
          button_style='info', # 'success', 'info', 'warning', 'danger' or ''
          tooltip=field,
          icon='check',  # (FontAwesome names without the `fa-` prefix)
          field=field
          )
     self.name = widgets.Text(
                    value=field,
                    placeholder='WB property name',
                    description='',
                    disabled=False
                  )

     self.example = widgets.Text(
                  value=str(csv.iloc[[2]][field]),
                  placeholder='WB property name',
                  description='',
                  disabled=False
                  )
     datatypes = ['select...','wikibase-item','url', 'string', 'monolingualtext', 'external-id', 'globe-coordinate', 'time']
     self.wb_datatype = widgets.Dropdown(options=datatypes, description='Select:')
  
   def on_button_clicked(self, b):
    if self.wb_datatype.value == "select...":
      b.button_style="danger"
    else:
      b.button_style='warning'
      prop_nr=createProperty(login,self.name.value, "", self.wb_datatype.value)
      b.description = prop_nr  
      properties[self.name]=prop_nr 

for field in list(csv):
  row = PropertyRow(field=field)
  grid[i,0] = row.PropertyButton
  grid[i,1] = row.name
  grid[i,2] = row.example
  grid[i,3] = row.wb_datatype
  i+=1

  continue



15


GridspecLayout(children=(Label(value='Add to Wikibase', layout=Layout(grid_area='widget001')), Label(value='Pr…

/usr/local/lib/python3.6/dist-packages/wikidataintegrator/wdi_core.py:219: UserWarning: Warning: No distinct value properties found
Please set P2302 and Q21502410 in your wikibase or set `core_props` manually.
Continuing with no core_props
  "Continuing with no core_props")
/usr/local/lib/python3.6/dist-packages/wikidataintegrator/wdi_core.py:175: UserWarning: mapping relation types are being ignored
  warnings.warn("mapping relation types are being ignored")


# Manually create properties
Often creating properties by replicating the field names of a table is not possible if an intended property combines multiple columns of a provided csv table. 

For example. Many table separate a coordinate location of a point on earth by its part, being the latitude, logitude and precision. Properties can be created manually on the wikibase itself, by selecting "*`create property`*" under the *`special pages`* link of a wikibase. 

